In [21]:
import pandas as pd
from flyvision_ans import NON_DECODING_CELLS, DECODING_CELLS
from scipy.sparse import load_npz

In [26]:
cc = pd.read_csv("adult_data/classification_clean.csv")
conns = pd.read_csv("adult_data/connections.csv")
neurons = pd.read_csv("adult_data/neurons.csv")
root_index = pd.read_csv("adult_data/root_id_to_index.csv")
synaptic_matrix = load_npz("adult_data/synaptic_matrix_sparse.npz")

In [17]:
# remove all the connections where pre_root_id is from a neuron whose cell type is not in the decoding cells
conns_cc = conns.merge(cc, left_on="pre_root_id", right_on="root_id")
conns_cc = conns_cc[~conns_cc["cell_type"].isin(NON_DECODING_CELLS)]

In [23]:
activation_root_ids = conns_cc[conns_cc["cell_type"].isin(DECODING_CELLS)]["pre_root_id"].unique()
activation_indices = root_index[root_index["root_id"].isin(
    activation_root_ids)]["index_id"].values

In [77]:
# create a vector of lentgh root_index.shape[0] with 0 everywhere except for the activation_indices where it is 1
activation_vector = pd.Series(0, index=root_index.index)
activation_vector[activation_indices] = 1

In [78]:
# simulate n passes of the activation_vector through the synaptic_matrix
import numpy as np


activated_neurons = activation_vector
n = 10
for i in range(n):
    activation_vector = synaptic_matrix.dot(activation_vector)# + activation_vector
    activated_neurons = activated_neurons + np.where(activation_vector > 0, 1, 0)
    print(activation_vector[activation_vector > 0].shape)

(50428,)
(87700,)
(111149,)
(126746,)
(127789,)
(125711,)
(79101,)
(64191,)
(63981,)
(63971,)


In [79]:
activated_neurons[activated_neurons > 0].shape

(127852,)

In [50]:
activation_vector.shape

(134191,)